In [1]:
# fundemental modules
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import time
import pprint
from tqdm.notebook import trange

from sac import Agent
from obs import observation_shape
from utils import record_videos, load_config

filename = 'inverted_pendulum.png'

best_score = -10000.0
best_score = -1000.0
score_history = []
episode_lens = []
avg_history = []
std_history = []
avg_history_100 = []




2023-12-10 22:31:10.874785: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-10 22:31:10.899395: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 22:31:10.899416: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 22:31:10.899433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 22:31:10.904036: I tensorflow/core/platform/cpu_feature_g

In [2]:
data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore
# pprint.pprint(env.config) # type: ignore
(obs, info), done = env.reset(), False

#observation config
proc = observation_shape(obs,info,2)
proc.reset()
input = proc.get_input()
print(input.shape)


(26,)


/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


In [3]:
agent = Agent(input_dims=input.shape, env=env,
            n_actions=1)

2023-12-10 22:31:17.584148: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-10 22:31:17.607584: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-10 22:31:17.607686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# main loop
for episode in trange(1000, desc='Test episodes'):

        proc.reset()        
        (observation, info), done = env.reset(), False
        truncated = False
        
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            action = agent.choose_action(observation)

            new_observation, reward, done, truncated, new_info = env.step(action=[action])
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >= 200:
                truncated = True

            agent.remember(state=observation, action=action, done=done,
                            reward=reward, new_state=new_observation)
            agent.learn()

            observation = new_observation

        episode_lens.append(episode_len)

        score_history.append(episode_reward)
        avg_score = np.mean(score_history)
        avg_history.append(avg_score)
        std_score = np.std(score_history)
        std_history.append(std_score)

        avg_score_100 = np.mean(score_history[-100:])
        avg_history_100.append(avg_score_100)

        agent.tensorboard.update_stats(episode_rew = episode_reward, 
                                       average_rew =avg_score,
                                       average_100_reward = avg_score_100,
                                       std_rew=std_score, 
                                       episode_len = episode_len)

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len,
              'avg score %.1f' % avg_score, 'avg_score_100 %.1f' %avg_score_100,'std score %.1f' % std_score)



Test episodes:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-12-10 22:31:51.703190: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


episode  0 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  1 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  2 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  3 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  4 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  5 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  6 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  7 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  8 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  9 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  10 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  11 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  12 score -0.3 ep len 5 avg score -0.3 avg

2023-12-10 22:31:54.535414: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e4a3b0b880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-10 22:31:54.535432: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-12-10 22:31:54.538125: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-10 22:31:55.435523: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-12-10 22:31:55.490312: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


episode  19 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  20 score -0.3 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  21 score -0.1 ep len 2 avg score -0.3 avg_score_100 -0.3 std score 0.0
episode  22 score 3.7 ep len 24 avg score -0.1 avg_score_100 -0.1 std score 0.8
episode  23 score 0.4 ep len 6 avg score -0.1 avg_score_100 -0.1 std score 0.8
episode  24 score 0.3 ep len 3 avg score -0.0 avg_score_100 -0.0 std score 0.8
episode  25 score 3.0 ep len 19 avg score 0.1 avg_score_100 0.1 std score 1.0
episode  26 score 3.5 ep len 8 avg score 0.2 avg_score_100 0.2 std score 1.2
episode  27 score 0.9 ep len 8 avg score 0.2 avg_score_100 0.2 std score 1.1
episode  28 score 0.9 ep len 9 avg score 0.2 avg_score_100 0.2 std score 1.1
episode  29 score 0.5 ep len 7 avg score 0.3 avg_score_100 0.3 std score 1.1
episode  30 score -0.3 ep len 3 avg score 0.2 avg_score_100 0.2 std score 1.1
episode  31 score 0.4 ep len 15 avg score 0.2 avg_score_10

KeyboardInterrupt: 

In [5]:
n_steps = sum(episode_lens)
print(n_steps)


3172


In [6]:

# env = record_videos(env)
from gymnasium.wrappers import RecordVideo

data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore

env = RecordVideo(env, video_folder="videos", episode_trigger=lambda e: True)
env.unwrapped.set_record_video_wrapper(env)


for episode in trange(1, desc='Test episodes'):

        proc.reset()        
        (observation, info), done = env.reset(), False
        truncated = False
        
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            action = agent.choose_action(observation)

            new_observation, reward, done, truncated, new_info = env.step(action=[action])
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1

            observation = new_observation

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len)

env.close()


/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(
/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/o/Documents/thesis/SAC_reward_Shaped/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Test episodes:   0%|          | 0/1 [00:00<?, ?it/s]

Moviepy - Building video /home/o/Documents/thesis/SAC_reward_Shaped/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/o/Documents/thesis/SAC_reward_Shaped/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/o/Documents/thesis/SAC_reward_Shaped/videos/rl-video-episode-0.mp4
episode  0 score 341.1 ep len 400
